In [ ]:
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import codecs
%matplotlib inline


Dataset from kaggle(dataset carino con anche delle sfide):

[kaggle dataset](https://www.kaggle.com/datasets/venky73/spam-mails-dataset?select=spam_ham_dataset.csv)

## Loading the dataset

In [ ]:
df = pd.read_csv("spam_ham_dataset.csv", index_col=0)

In [4]:
df

,label,text,label_num
605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
4685,spam,"Subject: photoshop , windows , office . cheap ...",1
2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
1409,ham,Subject: industrial worksheets for august 2000...,0


In [5]:
df.sample(5)

,label,text,label_num
420,ham,Subject: eops salary survey questionnaire\r\np...,0
823,ham,Subject: revised avails\r\n- - - - - - - - - -...,0
3793,spam,Subject: application declined\r\nare you drown...,1
1142,ham,Subject: revision # 1 - enron / hpl actuals fo...,0
1508,ham,Subject: re : new sitara desk request _ ena - ...,0


## è necessario pulire un po:
- tirare via gli spazi
- tirare via i \r \n

In [6]:
def remove_line_break(text):
    # Removes all line breaks
    for sign in '\r\n':
        text = text.replace(sign, " ")
    return text

In [7]:
df["text"]=df["text"].apply(remove_line_break) # applica a tutte le righe la funzione che passa come parametro

In [8]:
df.head()

,label,text,label_num
605,ham,Subject: enron methanol ; meter # : 988291 th...,0
2349,ham,"Subject: hpl nom for january 9 , 2001 ( see a...",0
3624,ham,"Subject: neon retreat ho ho ho , we ' re arou...",0
4685,spam,"Subject: photoshop , windows , office . cheap ...",1
2030,ham,Subject: re : indian springs this deal is to ...,0


In [9]:
df.shape

(5171, 3)

In [10]:
df.describe() # abbiamo più del 75 percento che sono email non di spam

,label_num
count,5171.000000
mean,0.289886
std,0.453753
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [11]:
df["label_num"].sum() # fornisce le email di spam

1499

## Using TF

### Unigrams

In [12]:
from sklearn.feature_extraction.text import CountVectorizer # prende ogni parloe e lo trasforme in un vettore
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None,max_features = 15000)
data_text = vectorizer.fit_transform(df["text"]) # analyzer = "word " -> analizzare per parole,  n_grams_range serve per settare il numero  di parole per ogni token, max_features = inserire un limite massimo di colonne da creare

In [13]:
data_text.toarray() # è una matrice  sparsa

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., 'zwallet', 'zxgwvpimicrosoft', 'zyban'],
      dtype=object)

In [15]:
vectorizer.get_feature_names_out()[1000:1010]

array(['579', '5796', '58', '580', '5801', '583', '5839', '584', '5843',
       '5848'], dtype=object)

In [16]:
vectorizer.get_feature_names_out()[10000:10010]

array(['multipart', 'multiple', 'multipliable', 'multiply', 'multitude',
       'mundo', 'munich', 'municipal', 'murder', 'murdered'], dtype=object)

## Possiamo allenare un modello

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)

In [19]:
from sklearn.naive_bayes import MultinomialNB
mnb= MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf=mnb.fit(X_train,y_train)

In [20]:
def accuracy(pred, result):
  return 1-(abs(pred-result)).sum()/len(result)

In [21]:
accuracy (mnb.predict(X_test),y_test)

0.9675174013921114

### Unigrams without stop words
tolgo le parole tipiche dell'inglese

In [22]:
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words="english",max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [23]:
data_text.shape

(5171, 15000)

In [24]:
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., 'zwiers', 'zxgwvpimicrosoft', 'zyban'],
      dtype=object)

In [28]:
vectorizer.get_feature_names_out()[10000:10010]

array(['nom 22', 'nom 30', 'nom actual', 'nom april', 'nom august',
       'nom change', 'nom daren', 'nom day', 'nom deal', 'nom dec'],
      dtype=object)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

0.9296210363495746

### Bigrams without stop words
lavoriamo con coppie di parole

In [26]:
vectorizer = CountVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words="english", ngram_range=(2,2),max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [27]:
vectorizer.get_feature_names_out()

array(['00 00', '00 000', '00 01', ..., 'zonedubai uae', 'zwallet com',
       'zyban prozac'], dtype=object)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

0.9327146171693735

### N-grams (1)
cambiando con char_wb -> suddivisione a livello di caratteri e non conta gli spazi
Però lavoriamo con colonne che non sono significative.

In [32]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None,max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [33]:
data_text.shape # 79 colonne

(5171, 79)

In [34]:
vectorizer.get_feature_names_out()

array(['\x01', '\x05', '\x07', '\x0f', '\x11', '\x12', '\x13', '\x14',
       '\x16', '\x1b', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')',
       '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']',
       '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
       'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w',
       'x', 'y', 'z', '{', '|', '}', '~'], dtype=object)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

0.7470997679814385

N-grams (2)
conta le coppie di caratteri

In [36]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None, ngram_range=(2,2), max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [37]:
data_text.shape

(5171, 976)

In [40]:
vectorizer.get_feature_names_out()[100:300]

array(['- ', '. ', '/ ', '0 ', '00', '01', '02', '03', '04', '05', '06',
       '07', '08', '09', '1 ', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '2 ', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '3 ', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '4 ', '40', '41', '42', '43', '44', '45', '46',
       '47', '48', '49', '5 ', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '6 ', '60', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '7 ', '70', '71', '72', '73', '74', '75', '76',
       '77', '78', '79', '8 ', '80', '81', '82', '83', '84', '85', '86',
       '87', '88', '89', '9 ', '90', '91', '92', '93', '94', '95', '96',
       '97', '98', '99', ': ', '; ', '< ', '= ', '> ', '? ', '??', '?a',
       '?b', '?c', '?d', '?e', '?f', '?g', '?l', '?m', '?n', '?o', '?p',
       '?r', '?s', '?t', '?v', '?w', '?x', '?y', '@ ', '[ ', '\\ ', '] ',
       '^ ', '_ ', '` ', 'a ', 'a?', 'aa', 'ab', '

In [41]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

0.8066511987625676

### N-grams (3)
con i 3-grammi

In [42]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None, ngram_range=(3,3), max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [43]:
data_text.shape

(5171, 15000)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

0.9319412219644239

In [ ]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None, ngram_range=(4,4), max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

### N-grams (5)
5-grammi

In [45]:
vectorizer = CountVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None, ngram_range=(5,5), max_features = 30000)
data_text = vectorizer.fit_transform(df["text"])

In [46]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

0.9644238205723125

## Cambio i  classificatori diversi

In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [48]:
import warnings
warnings.filterwarnings('ignore')

def classif_accuracy(clf,data_train, cat_train,test_data,test_cat):
    from sklearn.metrics import accuracy_score
       
    clf.fit(data_train, cat_train)
    pre=clf.predict(test_data)
    return accuracy_score(test_cat,pre) 

In [50]:
names = ["Decision Tree","Random Forest", "Naive Bayes", "Logistic Regression", "knn"]
classifiers = [
    DecisionTreeClassifier(max_depth=25),
    RandomForestClassifier(max_depth=25, n_estimators=100),
    MultinomialNB(),
    LogisticRegression(),
    KNeighborsClassifier()]

result_CV=pd.Series()

for name, clf in zip(names, classifiers):
    result_CV[name]=classif_accuracy(clf,X_train,y_train,X_test,y_test)

print(result_CV)
# logistic regression è buono

Decision Tree          0.917247
Random Forest          0.977572
Naive Bayes            0.964424
Logistic Regression    0.980665
knn                    0.899459
dtype: float64


## Using TF-IDF

### Unigrams
si utilizzano i numeri reali

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer = "word",tokenizer = None,preprocessor = None,stop_words = None,max_features = 15000)
data_text = vectorizer.fit_transform(df["text"])

In [52]:
data_text.toarray()

array([[0.10392772, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(data_text, df["label_num"], test_size=0.25)
clf=mnb.fit(X_train,y_train)
accuracy (mnb.predict(X_test),y_test)

0.9574632637277649

### N-grams (5)

In [55]:
vectorizer = TfidfVectorizer(analyzer = "char_wb",tokenizer = None,preprocessor = None,stop_words = None, ngram_range=(5,5), max_features = 30000)
data_text = vectorizer.fit_transform(df["text"])

In [56]:
names = ["Decision Tree","Random Forest", "Naive Bayes", "Logistic Regression", "knn"]
classifiers = [
    DecisionTreeClassifier(max_depth=25),
    RandomForestClassifier(max_depth=25, n_estimators=100),
    MultinomialNB(),
    LogisticRegression(),
    KNeighborsClassifier()]

result_CV=pd.Series()

for name, clf in zip(names, classifiers):
    result_CV[name]=classif_accuracy(clf,X_train,y_train,X_test,y_test)

print(result_CV) 

Decision Tree          0.950503
Random Forest          0.967517
Naive Bayes            0.957463
Logistic Regression    0.987626
knn                    0.969838
dtype: float64
